<a href="https://colab.research.google.com/github/Sai-sakunthala/hybrid-quantum-classical-algorithm/blob/main/term_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930

In [ ]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math

# Set up device
n_qubits = 2
n_shots = 5000
dev = qml.device("default.qubit", wires=n_qubits, shots=n_shots)

@qml.qnode(dev)
def bell_sampler():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

@qml.qnode(dev, interface="torch")
def bell_basis_sampler():
    # creating bell state
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])

    # computational basis changed to eigen basis
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=0)

    return qml.sample(wires=[0, 1])

def get_bell_samples():
    samples = bell_basis_sampler()
    return samples.to(torch.float32)

def get_custom_mixed_samples(n_samples=5000):
    samples = torch.zeros((n_samples, 2), dtype=torch.float32)
    probs = torch.rand(n_samples)
    samples[probs >= 0.7] = torch.tensor([1.0, 1.0])
    return samples

def get_uniform_samples(n_samples, n_bits):
    return torch.randint(0, 2, (n_samples, n_bits)).float()

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


In [ ]:
class EntropyNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

def parametric_qnee_cost(model, real_samples, uniform_samples):
    d = 2 ** real_samples.shape[1]
    term1 = torch.mean(model(real_samples))
    term2 = torch.mean(torch.exp(model(uniform_samples)))
    cost = -term1 + term2
    rel_entropy = 1 - cost
    entropy = np.log(d) - 1 + cost
    return cost, entropy, rel_entropy

##Bell state and maximally mixed state

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EntropyNet(input_dim=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(501):
    real_samples = get_bell_samples().to(device)
    uniform_samples = get_uniform_samples(len(real_samples), 2).to(device)

    loss, entropy, rel_entropy = parametric_qnee_cost(model, real_samples, uniform_samples)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Estimated von Neumann Entropy = {entropy.item():.4f}")
        print(f"Epoch {epoch}: Estimated relative entropy between bell state and maximally mixed state = {rel_entropy.item():.4f}")
        print('')

Epoch 0: Estimated von Neumann Entropy = 1.4303
Epoch 0: Estimated relative entropy between bell state and maximally mixed state = -0.0440

Epoch 100: Estimated von Neumann Entropy = 0.0988
Epoch 100: Estimated relative entropy between bell state and maximally mixed state = 1.2875

Epoch 200: Estimated von Neumann Entropy = 0.0452
Epoch 200: Estimated relative entropy between bell state and maximally mixed state = 1.3411

Epoch 300: Estimated von Neumann Entropy = 0.0375
Epoch 300: Estimated relative entropy between bell state and maximally mixed state = 1.3488

Epoch 400: Estimated von Neumann Entropy = 0.0177
Epoch 400: Estimated relative entropy between bell state and maximally mixed state = 1.3686

Epoch 500: Estimated von Neumann Entropy = 0.0045
Epoch 500: Estimated relative entropy between bell state and maximally mixed state = 1.3818



##Non uniform mixed state vs uniform mixed state

In [ ]:
for epoch in range(501):
    real_samples = get_custom_mixed_samples(n_samples=5000).to(device)
    uniform_samples = get_uniform_samples(len(real_samples), 2).to(device)

    loss, entropy, rel_entropy = parametric_qnee_cost(model, real_samples, uniform_samples)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Estimated entropy = {entropy.item():.4f}")
        print(f"Epoch {epoch}: Relative entropy vs uniform = {rel_entropy.item():.4f}")
        print('')

Epoch 0: Estimated entropy = 0.5957
Epoch 0: Relative entropy vs uniform = 0.7906

Epoch 100: Estimated entropy = 0.6244
Epoch 100: Relative entropy vs uniform = 0.7619

Epoch 200: Estimated entropy = 0.6080
Epoch 200: Relative entropy vs uniform = 0.7782

Epoch 300: Estimated entropy = 0.6087
Epoch 300: Relative entropy vs uniform = 0.7776

Epoch 400: Estimated entropy = 0.6340
Epoch 400: Relative entropy vs uniform = 0.7523

Epoch 500: Estimated entropy = 0.6179
Epoch 500: Relative entropy vs uniform = 0.7684



##Model complexity vs convergence

In [ ]:
# Define five models of increasing complexity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_configs = {
    "Linear Only": [
        nn.Linear(2, 1)
    ],
    "1 Hidden Layer (16 neurons)": [
        nn.Linear(2, 16),
        nn.ReLU(),
        nn.Linear(16, 1)
    ],
    "1 Hidden Layer (32 neurons)": [
        nn.Linear(2, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
    ],
    "1 Hidden Layer (64 neurons)": [
        nn.Linear(2, 64),
        nn.ReLU(),
        nn.Linear(64, 1)
    ],
    "2 Hidden Layers (64 → 32 neurons)": [  # Same as the last one, for consistency
        nn.Linear(2, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
    ],
}

# Training loop for each model
for name, layers in model_configs.items():
    print(f"\n--- Training {name} ---")

    model = nn.Sequential(*layers).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(501):
        real_samples = get_bell_samples().to(device)
        uniform_samples = get_uniform_samples(len(real_samples), 2).to(device)

        loss, entropy, rel_entropy = parametric_qnee_cost(model, real_samples, uniform_samples)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch 500: von Neumann Entropy = {entropy.item():.4f}")
    print(f"Epoch 500: Relative Entropy = {rel_entropy.item():.4f}")


--- Training Linear Only ---
Epoch 500: von Neumann Entropy = 0.9789
Epoch 500: Relative Entropy = 0.4074

--- Training 1 Hidden Layer (16 neurons) ---
Epoch 500: von Neumann Entropy = 0.0953
Epoch 500: Relative Entropy = 1.2910

--- Training 1 Hidden Layer (32 neurons) ---
Epoch 500: von Neumann Entropy = 0.0475
Epoch 500: Relative Entropy = 1.3388

--- Training 1 Hidden Layer (64 neurons) ---
Epoch 500: von Neumann Entropy = -0.0403
Epoch 500: Relative Entropy = 1.4266

--- Training 2 Hidden Layers (64 → 32 neurons) ---
Epoch 500: von Neumann Entropy = -0.0040
Epoch 500: Relative Entropy = 1.3903
